In [2]:
import os
import subprocess
import time
from datetime import datetime
import pandas as pd

# Load sentinel-1 product URLs from CSV
sentinel_ASF_products_urls = pd.read_csv('../data/sentinel_file_urls.csv')["product_url"].tolist()

print(sentinel_ASF_products_urls[1])

https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20231024T154105_20231024T154130_050909_0622FE_73C5.zip


In [3]:
# Define paths
aoi_path = "../utils/AOI_bboxes/aoi_shifa.geojson"
output_dir = "../data/preprocessed/sentinel"
snap_gpt_path = "/Applications/esa-snap/bin/gpt" 

# Create a dictionary to track processing results
results = []

# Process each URL
for i, url in enumerate(sentinel_ASF_products_urls):
    # Extract filename from URL
    filename = os.path.basename(url)

    # Extract date from filename
    date_str = filename.split('_')[5][:8]  # YYYYMMDD
    output_date_dir = os.path.join(output_dir, date_str)
    
    # Check if output for this date already exists and has files
    if os.path.exists(output_date_dir) and len(os.listdir(output_date_dir)) > 0:
        print(f"\n{'='*80}")
        print(f"Skipping file {i+1}/{len(sentinel_ASF_products_urls)}: {filename} (already processed)")
        print(f"{'='*80}\n")
        results.append({
            'filename': filename,
            'url': url,
            'success': True,
            'processing_time': 0,
            'date': date_str,
            'orbit': 'ASC' if 'T15' in filename else 'DESC',
            'skipped': True
        })
        continue  # Skip to next file
    
    # Print progress information
    print(f"\n{'='*80}")
    print(f"Processing file {i+1}/{len(sentinel_ASF_products_urls)}: {filename}")
    print(f"{'='*80}\n")
    
    # Track start time
    start_time = datetime.now()
    
    # Build command for download and processing
    cmd = [
        "python", 
        "../scripts/download_process_single_sentinel_file.py",  # Add the correct path to the script
        url, 
        aoi_path, 
        output_dir, 
        "--snap_gpt_path", 
        snap_gpt_path
    ]
    
    # Run the command
    try:
        # Execute download and processing
        process = subprocess.run(cmd, capture_output=False, text=True)
        success = process.returncode == 0
        end_time = datetime.now()
        processing_time = end_time - start_time
        
        # Record result
        results.append({
            'filename': filename,
            'url': url,
            'success': success,
            'processing_time': processing_time,
            'date': filename.split('_')[5][:8],  # Extract date YYYYMMDD
            'orbit': 'ASC' if 'T15' in filename else 'DESC'  # Determine orbit from time
        })
        
        # Store intermediate results in case of interruption
        pd.DataFrame(results).to_csv('../data/processing_results.csv', index=False)
        
        print(f"\nProcessed {filename} in {processing_time}")
        print(f"Status: {'SUCCESS' if success else 'FAILED'}")
        
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")
        results.append({
            'filename': filename,
            'url': url,
            'success': False,
            'processing_time': datetime.now() - start_time,
            'date': filename.split('_')[5][:8],
            'orbit': 'ASC' if 'T15' in filename else 'DESC',
            'error': str(e)
        })
        
    # Optional: Add a short delay between processing files
    time.sleep(1)

# Create a results DataFrame
results_df = pd.DataFrame(results)

# Calculate summary statistics
total_success = results_df['success'].sum()
total_failed = len(results_df) - total_success
total_time = results_df['processing_time'].sum()
avg_time = results_df['processing_time'].mean()

# Print summary
print("\n" + "="*80)
print("PROCESSING SUMMARY")
print("="*80)
print(f"Total files processed: {len(results_df)}")
print(f"Successfully processed: {total_success}")
print(f"Failed: {total_failed}")
print(f"Total processing time: {total_time}")
print(f"Average processing time: {avg_time}")
print("="*80)

# Save results to CSV
results_df.to_csv('../data/processing_results.csv', index=False)
print(f"Results saved to ../data/processing_results.csv")

# Create a Markdown-friendly table of results
from IPython.display import display, Markdown

# Format the results into a Markdown table
md_table = "| # | Filename | Orbit | Date | Success | Processing Time |\n"
md_table += "|---|---------|-------|------|---------|----------------|\n"

for i, row in results_df.iterrows():
    md_table += f"| {i+1} | {row['filename']} | {row['orbit']} | {row['date']} | {'✅' if row['success'] else '❌'} | {row['processing_time']} |\n"
    
display(Markdown(md_table))

# Plot success rate by orbit and time period
if len(results_df) > 0:
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        # Add reference/post classification based on date
        results_df['period'] = results_df['date'].apply(
            lambda x: 'Reference' if x < '20231007' else 'Post-conflict'
        )
        
        # Create a figure with summary plots
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Success by orbit
        orbit_success = results_df.groupby(['orbit', 'success']).size().unstack(fill_value=0)
        orbit_success.plot(kind='bar', stacked=True, ax=axes[0], 
                          color=['#ff9999', '#66b3ff'])
        axes[0].set_title('Processing Success by Orbit')
        axes[0].set_ylabel('Number of Files')
        
        # Success by period
        period_success = results_df.groupby(['period', 'success']).size().unstack(fill_value=0)
        period_success.plot(kind='bar', stacked=True, ax=axes[1],
                           color=['#ff9999', '#66b3ff'])
        axes[1].set_title('Processing Success by Period')
        axes[1].set_ylabel('Number of Files')
        
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error creating plots: {str(e)}")


Skipping file 1/54: S1A_IW_GRDH_1SDV_20231025T034436_20231025T034501_050916_062340_FA4F.zip (already processed)


Skipping file 2/54: S1A_IW_GRDH_1SDV_20231024T154105_20231024T154130_050909_0622FE_73C5.zip (already processed)


Skipping file 3/54: S1A_IW_GRDH_1SDV_20231013T034436_20231013T034501_050741_061D47_F325.zip (already processed)


Skipping file 4/54: S1A_IW_GRDH_1SDV_20231012T154104_20231012T154129_050734_061D05_072C.zip (already processed)


Skipping file 5/54: S1A_IW_GRDH_1SDV_20230919T034436_20230919T034501_050391_06114C_BDF8.zip (already processed)


Skipping file 6/54: S1A_IW_GRDH_1SDV_20230918T154105_20230918T154130_050384_06110A_47AC.zip (already processed)


Skipping file 7/54: S1A_IW_GRDH_1SDV_20230907T034436_20230907T034501_050216_060B4C_3E5C.zip (already processed)


Skipping file 8/54: S1A_IW_GRDH_1SDV_20230906T154104_20230906T154129_050209_060B15_3BB8.zip (already processed)


Skipping file 9/54: S1A_IW_GRDH_1SDV_20230814T034434_20230814T034459_049866_05FF61_BBF7

TypeError: unsupported operand type(s) for +: 'int' and 'datetime.timedelta'

 > Downloaded 1739909600 of 1739909600 bytes (100.00%)
Successfully downloaded 1739909600 bytes in 112.37 seconds
Average download rate: 14.77 MB/sec
Downloaded file: ../data/raw/sentinel/S1A_IW_GRDH_1SDV_20231105T154104_20231105T154129_051084_0628FB_34C7.zip
Download completed in 0:02:00.908924
File saved to: ../data/raw/sentinel/S1A_IW_GRDH_1SDV_20231105T154104_20231105T154129_051084_0628FB_34C7.zip
File size: 1659.31 MB
Preprocessing S1A_IW_GRDH_1SDV_20231105T154104_20231105T154129_051084_0628FB_34C7.zip
Using AOI: ../utils/AOI_bboxes/aoi_shifa.geojson
Output directory: ../data/preprocessed/sentinel
Running command: python ../scripts/preprocess_single_sentinel_zip.py ../data/raw/sentinel/S1A_IW_GRDH_1SDV_20231105T154104_20231105T154129_051084_0628FB_34C7.zip ../utils/AOI_bboxes/aoi_shifa.geojson ../data/preprocessed/sentinel --snap_gpt_path /Applications/esa-snap/bin/gpt
Preprocessing completed in 0:00:01.107675

Total processing time: 0:02:02.016909
Process failed.
